***
# Day 0/Initial Database Setup
1. Create a database in PostgresDB 
2. Load the data from CSV files for - 
    i. Inspections table
    ii. violations table

The csv files are expected to be stored in /Data location under parent directory
The csv files are expected to have consistent headers.

Further process the data to -
1. create the aggregate from violations table
2. Join the datasets to get the number of violations for every facility
***

In [2]:
import toml
import os
import pandas as pd

In [3]:
#Set Pandas options
pd.set_option('display.max_columns', None)

In [4]:
#import generic functions for database operations
from create_database_function import create_postgres_db
from create_table_function import create_postgres_table
from create_table_from_pandas_df_function import create_table_from_df
from create_df_from_sql_function import create_df_from_sql
from day0_data_load import load_data_day0

In [5]:
#get current working directory
curr_dir = os.getcwd()
print(curr_dir)

C:\Users\guest1\MSc_Advanced_Programming\advanced-programming-assessment\database_operations


In [6]:
os.chdir('..')
wdir = os.getcwd()
print(wdir)

C:\Users\guest1\MSc_Advanced_Programming\advanced-programming-assessment


In [7]:
config = toml.load(wdir + "\config.toml")
print (config)

{'postgres_credentials': {'username': 'postgres', 'password': 'admin', 'host': '127.0.0.1', 'port': '5432'}, 'db_details': {'dbname': 'inspections_db', 'inspections_table': 'inspections', 'violations_table': 'violations', 'violations_agg_table': 'violations_aggregated', 'inspections_enriched_table': 'inspections_enriched', 'inspections_cleaned_table': 'inspections_cleaned', 'zip_statistics_table': 'statistics_zip_code', 'seating_type_statistics_table': 'statistics_seating_type', 'seating_capacity_statistics_table': 'statistics_seating_capacity', 'viz_violation_type': 'viz_violation_type', 'viz_num_violations': 'viz_violation_type', 'viz_geography': 'viz_geography'}}


***
Create a database in postgres
***

In [8]:
#Call create database function to create a new database in postgres called 'inspection_db'
create_postgres_db(config['postgres_credentials'], config['db_details']['dbname'])

Database already exists:  inspections_db
PostgreSQL connection is closed


***
Create a Table for inspections data
***

In [9]:
#Load data in Pandas dataframe so as to get the datatypes standardised

inspections = pd.read_csv(wdir + '\Data\Inspections.csv')

In [10]:
list(inspections.columns)

['ACTIVITY DATE',
 'OWNER ID',
 'OWNER NAME',
 'FACILITY ID',
 'FACILITY NAME',
 'RECORD ID',
 'PROGRAM NAME',
 'PROGRAM STATUS',
 'PROGRAM ELEMENT (PE)',
 'PE DESCRIPTION',
 'FACILITY ADDRESS',
 'FACILITY CITY',
 'FACILITY STATE',
 'FACILITY ZIP',
 'SERVICE CODE',
 'SERVICE DESCRIPTION',
 'SCORE',
 'GRADE',
 'SERIAL NUMBER',
 'EMPLOYEE ID',
 'Location',
 '2011 Supervisorial District Boundaries (Official)',
 'Census Tracts 2010',
 'Board Approved Statistical Areas',
 'Zip Codes']

In [11]:
#Rename dataframe columns in order to be standardised for Postgres schema
inspections_col_rename = {
    'ACTIVITY DATE':'activity_date', 
    'OWNER ID':'owner_id', 
    'OWNER NAME':'owner_name', 
    'FACILITY ID':'facility_id',
    'FACILITY NAME':'facility_name', 
    'RECORD ID':'record_id', 
    'PROGRAM NAME':'program_name', 
    'PROGRAM STATUS':'program_status',
    'PROGRAM ELEMENT (PE)':'program_element_pe', 
    'PE DESCRIPTION':'pe_desc', 
    'FACILITY ADDRESS':'facility_address',
    'FACILITY CITY':'facility_city', 
    'FACILITY STATE':'facility_state', 
    'FACILITY ZIP':'facility_zip', 
    'SERVICE CODE':'service_code',
    'SERVICE DESCRIPTION':'service_desc', 
    'SCORE':'score', 
    'GRADE':'grade', 
    'SERIAL NUMBER':'serial_number', 
    'EMPLOYEE ID':'employee_id',
    'Location':'location', 
    '2011 Supervisorial District Boundaries (Official)':'supervisorial_district_boundaries',
    'Census Tracts 2010':'census_tracts_2010', 
    'Board Approved Statistical Areas':'board_approved_statistical_areas', 
    'Zip Codes':'zip_code'
    }

inspections = inspections.rename(columns=inspections_col_rename)

In [10]:
#Create a table for inspections using a generic function
create_table_from_df(config['postgres_credentials'], config['db_details']['dbname'], config['db_details']['inspections_table'], inspections)

Table successfully created: inspections


***
Create a Table for violations data
***

In [16]:
#Load data in Pandas dataframe so as to get the datatypes standardised

violations = pd.read_csv(wdir + "\Data" + "\\" + "violations.csv")

In [17]:
list(violations.columns)

['SERIAL NUMBER',
 'VIOLATION  STATUS',
 'VIOLATION CODE',
 'VIOLATION DESCRIPTION',
 'POINTS']

In [12]:
violations.head(5)

,SERIAL NUMBER,VIOLATION STATUS,VIOLATION CODE,VIOLATION DESCRIPTION,POINTS
0,DA2FXQNN6,OUT OF COMPLIANCE,F048,# 47. Permits Available,1
1,DA2FXQNN6,OUT OF COMPLIANCE,F046,# 46. Signs posted; last inspection report ava...,1
2,DA2FXQNN6,OUT OF COMPLIANCE,F033,# 33. Nonfood-contact surfaces clean and in go...,1
3,DACP43IQW,OUT OF COMPLIANCE,F048,# 47. Permits Available,1
4,DACP43IQW,OUT OF COMPLIANCE,F044,"# 44. Floors, walls and ceilings: properly bui...",1


In [13]:
#Rename dataframe columns in order to be standardised for Postgres schema
violations_col_rename = {
    'SERIAL NUMBER':'serial_number', 
    'VIOLATION STATUS':'violation_status', 
    'VIOLATION CODE':'violation_code',
    'VIOLATION DESCRIPTION':'violation_desc', 
    'POINTS':'points'
    }

violations = violations.rename(columns=violations_col_rename)

In [14]:
violations.head(5)

,serial_number,VIOLATION STATUS,violation_code,violation_desc,points
0,DA2FXQNN6,OUT OF COMPLIANCE,F048,# 47. Permits Available,1
1,DA2FXQNN6,OUT OF COMPLIANCE,F046,# 46. Signs posted; last inspection report ava...,1
2,DA2FXQNN6,OUT OF COMPLIANCE,F033,# 33. Nonfood-contact surfaces clean and in go...,1
3,DACP43IQW,OUT OF COMPLIANCE,F048,# 47. Permits Available,1
4,DACP43IQW,OUT OF COMPLIANCE,F044,"# 44. Floors, walls and ceilings: properly bui...",1


In [15]:
#Create a table for inspections using a generic function
create_table_from_df(config['postgres_credentials'], config['db_details']['dbname'], config['db_details']['violations_table'], violations)

Table successfully created: violations


***
Aggregate violations data to get number of violations per serial number
***

In [16]:
#Using create_df_from_sql function create the pandas dataframe for violations table.
#Pass connection config, dbname and table name to the function
violations_data = create_df_from_sql(config['postgres_credentials'], config['db_details']['dbname'], config['db_details']['violations_table'])

In [17]:
violations_data.head(5)

,serial_number,VIOLATION STATUS,violation_code,violation_desc,points
0,DA2FXQNN6,OUT OF COMPLIANCE,F048,# 47. Permits Available,1
1,DA2FXQNN6,OUT OF COMPLIANCE,F046,# 46. Signs posted; last inspection report ava...,1
2,DA2FXQNN6,OUT OF COMPLIANCE,F033,# 33. Nonfood-contact surfaces clean and in go...,1
3,DACP43IQW,OUT OF COMPLIANCE,F048,# 47. Permits Available,1
4,DACP43IQW,OUT OF COMPLIANCE,F044,"# 44. Floors, walls and ceilings: properly bui...",1


In [18]:
#Get the type of violation for every record
char1 = '#'
char2 = '.'
violations_data['violation_type'] = violations_data['violation_desc'].apply(lambda x: x[x.find(char1)+1: x.find(char2)].strip())

In [19]:
violations_data.head(5)

,serial_number,VIOLATION STATUS,violation_code,violation_desc,points,violation_type
0,DA2FXQNN6,OUT OF COMPLIANCE,F048,# 47. Permits Available,1,47
1,DA2FXQNN6,OUT OF COMPLIANCE,F046,# 46. Signs posted; last inspection report ava...,1,46
2,DA2FXQNN6,OUT OF COMPLIANCE,F033,# 33. Nonfood-contact surfaces clean and in go...,1,33
3,DACP43IQW,OUT OF COMPLIANCE,F048,# 47. Permits Available,1,47
4,DACP43IQW,OUT OF COMPLIANCE,F044,"# 44. Floors, walls and ceilings: properly bui...",1,44


In [20]:
violations_aggregated = violations_data\
.groupby('serial_number')\
.agg({'violation_desc':'count', 'violation_type':lambda x: list(x)})\
.reset_index()\
.rename(columns={'violation_desc':'num_violations'})

In [21]:
violations_aggregated.head(5)

,serial_number,num_violations,violation_type
0,DA1HW0SXT,5,"[01b, 42, 39, 38, 07]"
1,DA2BIJ7IY,1,[14]
2,DA2BJJBJV,3,"[39, 35, 33]"
3,DA2FXQNN6,3,"[47, 46, 33]"
4,DA3QBCPMJ,4,"[01b, 36, 30, 25]"


In [22]:
violations_aggregated['serial_number'].count() == violations_aggregated['serial_number'].nunique()

True

In [23]:
#Write back the violations aggregated data to postgres
create_table_from_df(config['postgres_credentials'], config['db_details']['dbname'], config['db_details']['violations_agg_table'], violations_aggregated)

Table successfully created: violations_aggregated


***
Join Inspections with violations aggregated to enrich the inspections data
***

In [24]:
#Using create_df_from_sql function create the pandas dataframe for inspections table.
#Pass connection config, dbname and table name to the function
inspections_data = create_df_from_sql(config['postgres_credentials'], config['db_details']['dbname'], config['db_details']['inspections_table'])

In [25]:
inspections_data.head(5)

,activity_date,owner_id,owner_name,facility_id,facility_name,record_id,program_name,program_status,program_element_pe,pe_desc,facility_address,facility_city,facility_state,facility_zip,service_code,service_desc,score,grade,serial_number,employee_id,location,supervisorial_district_boundaries,census_tracts_2010,board_approved_statistical_areas,zip_code
0,08/23/2018,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,97,A,DA2FXQNN6,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719
1,12-06-17,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,95,A,DACP43IQW,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719
2,06/23/2017,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,96,A,DAEMVMRBY,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719
3,03/19/2019,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,96,A,DANER68S4,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719
4,03-01-18,OW0000002,#1 CAFE INC,FA0056432,#1 CAFE,PR0045100,#1 CAFE,ACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,2080 CENTURY PARK E STE 108,LOS ANGELES,CA,90067,1,ROUTINE INSPECTION,90,A,DACZXQ74W,EE0000015,POINT (-118.412323 34.058815),3,660,265,24032


In [26]:
inspections_data.count()

activity_date                        25
owner_id                             25
owner_name                           25
facility_id                          25
facility_name                        25
record_id                            25
program_name                         25
program_status                       25
program_element_pe                   25
pe_desc                              25
facility_address                     25
facility_city                        25
facility_state                       25
facility_zip                         25
service_code                         25
service_desc                         25
score                                25
grade                                25
serial_number                        25
employee_id                          25
location                             25
supervisorial_district_boundaries    25
census_tracts_2010                   25
board_approved_statistical_areas     25
zip_code                             25


In [27]:
#Using create_df_from_sql function create the pandas dataframe for violations aggregated table.
#Pass connection config, dbname and table name to the function
violations_agg_data = create_df_from_sql(config['postgres_credentials'], config['db_details']['dbname'], config['db_details']['violations_agg_table'])

In [28]:
violations_agg_data.head(5)

,serial_number,num_violations,violation_type
0,DA1HW0SXT,5,"{01b,42,39,38,07}"
1,DA2BIJ7IY,1,{14}
2,DA2BJJBJV,3,"{39,35,33}"
3,DA2FXQNN6,3,"{47,46,33}"
4,DA3QBCPMJ,4,"{01b,36,30,25}"


In [29]:
#Join 2 datasets to enrich inspections data with number of violations
inspections_enriched = pd\
.merge(inspections_data, violations_agg_data, on='serial_number', how='inner')

In [30]:
inspections_enriched.head(5)

,activity_date,owner_id,owner_name,facility_id,facility_name,record_id,program_name,program_status,program_element_pe,pe_desc,facility_address,facility_city,facility_state,facility_zip,service_code,service_desc,score,grade,serial_number,employee_id,location,supervisorial_district_boundaries,census_tracts_2010,board_approved_statistical_areas,zip_code,num_violations,violation_type
0,08/23/2018,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,97,A,DA2FXQNN6,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,3,"{47,46,33}"
1,12-06-17,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,95,A,DACP43IQW,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,5,"{47,44,40,35,34}"
2,06/23/2017,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,96,A,DAEMVMRBY,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,4,"{37,35,34,33}"
3,03/19/2019,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,96,A,DANER68S4,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,4,"{44,43,40,33}"
4,03-01-18,OW0000002,#1 CAFE INC,FA0056432,#1 CAFE,PR0045100,#1 CAFE,ACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,2080 CENTURY PARK E STE 108,LOS ANGELES,CA,90067,1,ROUTINE INSPECTION,90,A,DACZXQ74W,EE0000015,POINT (-118.412323 34.058815),3,660,265,24032,9,"{50,40,36,35,34,33,27,14,07}"


In [31]:
inspections_enriched.count()

activity_date                        25
owner_id                             25
owner_name                           25
facility_id                          25
facility_name                        25
record_id                            25
program_name                         25
program_status                       25
program_element_pe                   25
pe_desc                              25
facility_address                     25
facility_city                        25
facility_state                       25
facility_zip                         25
service_code                         25
service_desc                         25
score                                25
grade                                25
serial_number                        25
employee_id                          25
location                             25
supervisorial_district_boundaries    25
census_tracts_2010                   25
board_approved_statistical_areas     25
zip_code                             25


In [32]:
#Write inspections enriched data to postgres table
create_table_from_df(config['postgres_credentials'], config['db_details']['dbname'], config['db_details']['inspections_enriched_table'], inspections_enriched)

Table successfully created: inspections_enriched


***
Test load_data_day0 function
***

In [12]:
os.chdir(curr_dir)

In [13]:
load_data_day0()

Current Directory is: C:\Users\guest1\MSc_Advanced_Programming\advanced-programming-assessment\database_operations
Current Directory is: C:\Users\guest1\MSc_Advanced_Programming\advanced-programming-assessment
Config file loaded successfully.
Creating database in Postgres.
Database already exists:  inspections_db
PostgreSQL connection is closed
Reading inspections data from raw csv file.
Renaming columns of inspections data to standard nomenclature for Postgres.
Creating a table in Postgres DB and loading data for inspections.
Table successfully created: inspections
Data load for inspections data is completed.
Reading violations data from raw csv file.
Renaming columns of violations data to standard nomenclature for Postgres.
Creating a table in Postgres DB and loading data for violations.
Table successfully created: violations
Data load for violations data is completed.
Reading violations data from Postgres DB.
Aggregating violations data to get number of violations.
Creating a table 

True